In [34]:
import pandas as pd
import numpy as np
import pickle

with open('./science_x_pro.pkl', 'rb') as f:
    science_x = pickle.load(f)

In [10]:
with open('./science_y.pkl', 'rb') as f:
    science_y = pickle.load(f)

In [35]:
with open('./social_x_pro.pkl', 'rb') as f:
    social_x = pickle.load(f)

In [12]:
with open('./social_y.pkl', 'rb') as f:
    social_y = pickle.load(f)

In [95]:
from collections import OrderedDict
mydict = {}

for i in science_x:
    for j in set(i):
        if j in mydict:
            mydict[j] = mydict[j] + 1
        else:
            mydict[j] = 1
OrderedDict(sorted(mydict.items(), key=lambda t: t[0]))

dictionary = pd.DataFrame.from_dict(mydict, orient='index').reset_index().rename(columns={'index': 'term', 0: 'df'})

n = len(science_x)
idf = OrderedDict()

for key in mydict.keys():
    idf[key] = np.log10(n / mydict[key])

index = 0
tfs = []

for i in science_x:
    index += 1
    tf = {}
    for j in i:
        if j in tf:
            tf[j] = tf[j] + 1
        else:
            tf[j] = 1
    tfs.append(tf)

tf_idf = pd.DataFrame(0.0, index=np.arange(n), columns = dictionary['term'])
index = 0

for tf in tfs:
    index += 1
    for key in tf.keys():
        tf_idf[key][index-1] =  tf[key] * idf[key]
        
tf_idf

term,綠色,人類,動物,地球,生存,太陽,大部分,生命,生物,毀滅,...,快用,燒杯,早餐,一盞,阿東,正拿著,燈準備,嬉鬧並,加熱並,做法
0,1.763071,5.010952,1.299961,2.085291,1.971347,2.307139,1.939163,1.593929,1.009744,3.085291,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.593929,1.009744,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.187858,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,1.971347,0.000000,0.000000,1.593929,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.593929,1.009744,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1213,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1215,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.085291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [99]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

x = SelectKBest(chi2, k=1000).fit_transform(tf_idf, science_y)

In [103]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, science_y, test_size=0.2)

In [104]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='macro'))

0.8565573770491803
0.8196103073576488
0.8410025773394757
0.8263150437819333


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [105]:
mydict = {}

for i in social_x:
    for j in set(i):
        if j in mydict:
            mydict[j] = mydict[j] + 1
        else:
            mydict[j] = 1
OrderedDict(sorted(mydict.items(), key=lambda t: t[0]))

dictionary = pd.DataFrame.from_dict(mydict, orient='index').reset_index().rename(columns={'index': 'term', 0: 'df'})

n = len(social_x)
idf = OrderedDict()

for key in mydict.keys():
    idf[key] = np.log10(n / mydict[key])

index = 0
tfs = []

for i in social_x:
    index += 1
    tf = {}
    for j in i:
        if j in tf:
            tf[j] = tf[j] + 1
        else:
            tf[j] = 1
    tfs.append(tf)

tf_idf = pd.DataFrame(0.0, index=np.arange(n), columns = dictionary['term'])
index = 0

for tf in tfs:
    index += 1
    for key in tf.keys():
        tf_idf[key][index-1] =  tf[key] * idf[key]
        
tf_idf

term,目和舉行,孝廉,指定,禮樂,九品,描述,分成,禮,經由,集體,...,教學,譯述將,好感,超林樂知,解放,問兩者,改信,灣社會,剪除,投票
0,2.835691,1.881448,2.835691,1.835691,1.493268,1.556937,2.358569,2.358569,2.136721,2.835691,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,1.835691,1.493268,0.000000,0.000000,2.358569,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,1.556937,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
681,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
683,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.835691,2.835691,2.835691,2.835691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [110]:
x = SelectKBest(chi2, k=1000).fit_transform(tf_idf, social_y)
X_train, X_test, y_train, y_test = train_test_split(x, social_y, test_size=0.2)

In [111]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='macro'))

0.8686131386861314
0.9035900090247917
0.8709892290249434
0.8812986573309189
